# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [34]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import AzureOpenAI

In [35]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
api_version = os.getenv('OPENAI_API_VERSION')
azure_endpoint = os.getenv('OPENAI_ENDPOINT')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook!")
elif not api_version:
    print("No API version found - please add OPENAI_API_VERSION to your .env file")
elif not azure_endpoint:
    print("No Azure endpoint found - please add OPENAI_ENDPOINT to your .env file")
else:
    print("Azure OpenAI credentials found and look good!")
    
MODEL = 'gpt-4o'  # Your Azure deployment name
openai = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
)

Azure OpenAI credentials found and look good!


In [36]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [37]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [38]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [39]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [40]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [41]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [42]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'branding page', 'url': 'https://huggingface.co/brand'}]}

In [43]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [44]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-4o
Found 2 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum'}]}

In [45]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4o
Found 2 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/docs'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [46]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [47]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-4o
Found 3 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-4.7-Flash
Updated
1 day ago
•
69.5k
•
922
zai-org/GLM-Image
Updated
7 days ago
•
9.87k
•
936
google/translategemma-4b-it
Updated
7 days ago
•
43.2k
•
457
kyutai/pocket-tts
Updated
3 days ago
•
36.5k
•
386
nvidia/personaplex-7b-v1
Updated
1 day ago
•
2.43k
•
305
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.04k
Qwen Image Multiple Angles 3D Camera
🎥
1.04k
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.69k
Z Image Turbo
🖼
1.69k
Generate stunning images from text descriptions in seconds
Ru

In [48]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [49]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [50]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4o
Found 4 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-4.7-Flash\nUpdated\n1 day ago\n•\n69.5k\n•\n922\nzai-org/GLM-Image\nUpdated\n7 days ago\n•\n9.87k\n•\n936\ngoogle/translategemma-4b-it\nUpdated\n7 days ago\n•\n43.2k\n•\n457\nkyutai/pocket-tts\nUpdated\n3 days ago\n•\n36.5k\n•\n386\nnvidia/personaplex-7b-v1\nUpdated\n1 day ago\n•\n2.43k\n•\n305\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n1.04k\nQwen Image Multiple A

In [51]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [52]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4o
Found 4 relevant links


# Hugging Face: Leading the Future of AI  

## Who We Are  
Hugging Face is a vibrant AI platform and community that is building the future of machine learning (ML). As the home of over 2 million ML models, 1 million applications, and half a million datasets, we provide a collaborative space for developers, researchers, and organizations to create, discover, and share AI-powered innovations. Our ecosystem empowers the ML community to explore and grow through cutting-edge tools across text, image, video, audio, and even 3D.

## What We Do  
Hugging Face's platform allows machine learning enthusiasts to:  
- Collaborate on unlimited public ML models, datasets, and applications.  
- Accelerate their projects using open-source tools and frameworks.  
- Build individual or organizational portfolios to showcase their work.  

We also cater to major businesses with enterprise-grade AI solutions, offering enhanced security, advanced analytics, and scalable resources tailored to organizational needs.

## For ML Enthusiasts and Developers  
Hugging Face empowers individuals to:  
- Explore vast libraries of open ML models and datasets.  
- Build applications using state-of-the-art tools within a supportive community.  
- Learn through a suite of educational courses such as Large Language Models (LLMs), Robotics, Deep Reinforcement Learning, and Computer Vision.  
- Grow professionally via a collaborative ecosystem that highlights their contributions to the ML community.  

## Enterprise Solutions  
For companies, we deliver AI capabilities with comprehensive tools for scaling and security:  
- Enterprise-grade security features like Single Sign-On, granular access control, audit logs, and token management.  
- Advanced compute options, enhanced storage, and private dataset viewing to optimize collaboration.  
- Inference providers and managed billing to oversee costs efficiently.  
- Dedicated support and analytics to ensure smooth platform adoption and use.  
Our enterprise plans are designed to fit organizations of all sizes, starting from $20/user/month or providing flexible custom options.  

## Dynamic Community  
Hugging Face thrives as a leading-edge AI hub with a diverse community of researchers, developers, and innovators who collaborate to drive machine learning forward. From AI agents to deep learning experiments, our platform nurtures both individual talent and collective progress.  

## Educational Offerings  
We believe in empowering the next generation of researchers, developers, and AI enthusiasts. Hugging Face provides a variety of learning opportunities such as:  
- **LLM Course**: Dive into Large Language Models using Hugging Face libraries.  
- **Deep RL Course**: Explore deep reinforcement learning methods.  
- **Robotics Course**: Learn to build robots from the ground up.  
- **AI Agents Course**: Develop and deploy your own AI agents.  

## A Word to Collaborators  
Be a part of the Hugging Face journey! Whether you're a researcher looking to share innovative models, a business solving complex challenges with AI, or a developer eager to learn, grow, and contribute, Hugging Face offers the tools and community to bring your vision to life.

Explore the future of AI with us by visiting our platform, exploring models and datasets, or joining our community. Together, let’s push the boundaries of what AI can achieve!  

### Discover More  
Website: [Hugging Face](https://huggingface.co)  
Sign Up and Begin Your Journey Today!  

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [59]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:
            response += chunk.choices[0].delta.content
            update_display(Markdown(response), display_id=display_handle.display_id)

In [60]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4o
Found 5 relevant links


# Discover Hugging Face  
### The AI Community Building the Future  

Welcome to Hugging Face, the premier collaboration platform for machine learning (ML). As the heart of the ML community, Hugging Face empowers engineers, scientists, organizations, and enthusiasts to create, discover, and collaborate on the next generation of artificial intelligence.  

### What We Do  

Hugging Face offers an all-in-one platform to:  
- **Host and Share:** Access 2M+ models, 1M+ ML applications, and 500K+ datasets.  
- **Collaborate:** Work with the community on open-source projects like models or datasets in text, image, video, audio, and 3D domains.  
- **Accelerate AI Innovation:** Utilize advanced tools such as ZeroGPU technology and an open-source ML stack to speed up development.  
- **Build Portfolios:** Share your projects with a global audience and enhance your professional profile.  

### Perfect for Individuals and Enterprises  

Whether you're an independent developer or a large-scale organization, Hugging Face offers flexible solutions:  
- **Individuals:** Collaborate publicly for free, develop your skills, and contribute to an open AI-driven future.  
- **Enterprise Support:** With plans starting at $20/user/month, companies gain access to advanced features like single sign-on (SSO), resource management, private datasets, analytics, and priority support to scale seamlessly.  

### Our Mission  

Hugging Face’s mission is to enable collaborative, open, and ethical AI development. By bringing together the brightest and most creative minds, we build tools for a more transparent and equitable AI landscape that benefits everyone.  

### Why Hugging Face?  

- **Scalability and Performance:** Enterprise-grade features such as advanced compute options, ZeroGPU quota boosts, and enhanced storage options.  
- **Community-Driven:** The Hugging Face Hub is a vibrant ML ecosystem where the global community innovates daily.  
- **Secure Collaboration:** Featuring advanced security policies, granular access controls, and private storage options.  
- **Ethics and Openness:** Hugging Face promotes transparency and ethical AI practices, making it a trusted platform for responsible innovation.  

### Join the AI Revolution  

**Careers at Hugging Face**: Dream of working alongside AI innovators? Hugging Face fosters a welcoming, creative, and inclusive work culture aimed at creating positive global impact. Check out our current openings on our careers page to join the team that's shaping the future of AI.  

### Explore Hugging Face  

Sign up today and dive into a massive ecosystem of models, datasets, and applications. Whether you're an ML enthusiast, researcher, or enterprise leader, Hugging Face is the ultimate platform to explore and excel in AI development.  

For more information, visit our platform or contact sales for custom enterprise solutions.  

Together, we build the future of AI.

In [61]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4o
Found 3 relevant links


# Hugging Face: Building the Future of AI  

### About Hugging Face  
Hugging Face is the leading collaboration platform for the machine learning (ML) community. It serves as a central hub where engineers, scientists, enthusiasts, and organizations come together to share, explore, and innovate using open-source ML models, datasets, and applications.  

With a strong mission to empower the next generation of machine learning professionals, Hugging Face builds a community-driven approach to advance open and ethical AI development. Whether you're an individual developer, a researcher, or part of an enterprise, Hugging Face provides the tools to create, discover, and collaborate better in the world of machine learning.  

---

### Why Choose Hugging Face?  

#### **For Machine Learning Enthusiasts**  
- **A Rich Repository**: Access 2M+ ML models, 1M+ AI applications, and over 500k datasets covering text, images, audio, video, and even 3D.  
- **A Collaborative Platform**: Share your work, build your portfolio, and engage with a rapidly growing community of innovators.  
- **Open-Source Focus**: Leverage the HF Open Source stack to accelerate your learning and projects.  

#### **For Enterprises**  
- **Scalable Solutions**: Hugging Face Enterprise offers advanced tools for businesses, including enterprise-grade security, private storage, and powerful compute options like ZeroGPU.  
- **Enhanced Collaboration**: Manage access, analytics, and repository locations with resource groups, audit logs, and centralized token management.  
- **Premium Support**: Gain priority support and keep your teams moving faster with the most advanced platform.  
- **Flexible Pricing**: Team plans start at $20/user/month, with tailored options for enterprise customers.  

#### **For the ML Future**  
Hugging Face focuses on driving ethical AI. Its open-source ethos encourages transparency, accessibility, and inclusion, ensuring a robust ecosystem for diverse ML applications.

---

### Hugging Face Community  
Hugging Face is more than just a platform—it's a vibrant AI community. By joining, you can:  
- Collaborate on cutting-edge AI models and datasets.  
- Stay updated on the newest trends and best practices in machine learning.  
- Explore practical applications for industries ranging from healthcare to creative arts.  
- Contribute ideas and code to shape the future of AI collaboratively.  

---

### Career Opportunities at Hugging Face  
Hugging Face values innovation, community, and passion for technology.  
- **Join a Fast-Growing AI Startup**: Work at the forefront of machine learning collaboration and research.  
- **Contribute to Ethical AI**: Help empower the next generation of engineers, scientists, and end users by building an inclusive and open AI ecosystem.  
- **Collaborative Culture**: Become part of a team that champions innovation, transparency, and shared knowledge.  

If you are excited about shaping the future of AI, Hugging Face offers an inspiring platform to grow your skills and make a meaningful impact.

---

### For Enterprises  
Bolster your AI potential with Hugging Face's enterprise-grade solutions.  
- Scale efficiently and securely.  
- Gain advanced data, analytics, and compute tools.  
- Manage your budget effectively with clear billing options.  
Contact Hugging Face sales to explore flexible contract options tailored to your organization's needs.

---

### Explore & Get Started  
From individual contributors to leading enterprises, Hugging Face provides everyone access to tools for building the future.  
- **Discover Amazing Models**: Browse over 2M+ models and trending applications.  
- **Experiment with Datasets**: Access a rich library of datasets for training and research.  
- **Join for Free**: Sign up today and explore all the possibilities of the Hugging Face platform.  

#### Website: [Hugging Face](https://huggingface.co)  

Hugging Face is where technology meets collaboration, turning groundbreaking ideas into reality and creating a better future for AI. Come and be part of the transformation. 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>